In [1]:
import pandas as pd
import numpy as np

In [2]:
app_events = pd.read_csv('../talkingdata_DATA/app_events.csv')

In [46]:
brand_df = pd.read_csv('../talkingdata_DATA/phone_brand_device_model.csv')

In [3]:
label_categories = pd.read_csv('../talkingdata_DATA/label_categories.csv')
app_labels = pd.read_csv('../talkingdata_DATA/app_labels.csv')
events = pd.read_csv('../talkingdata_DATA/events.csv')

In [4]:
gender_age_train = pd.read_csv('../talkingdata_DATA/gender_age_train.csv')
gender_age_test = pd.read_csv('../talkingdata_DATA/gender_age_test.csv')

In [6]:
display(gender_age_train.head()) # want to predict group, involves multiclass classification

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


### Clean app_events and events dataframes: 

In [8]:
display(app_events.head()) # event_id corresponds to that in events dataframe
display(events.head())

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [14]:
# convert timestamp column to pd.datetime
events['timestamp'] = events['timestamp'].map(pd.to_datetime)

### Clean phone brand dataframe (brand_df)

In [47]:
# dictionary of brand categories and their associated numbers/indices
brand_categories = brand_df[['phone_brand']].drop_duplicates().reset_index(drop=True).reset_index()
brand_categories = brand_categories.rename(columns = {'index': 'phone_brand_id'})

In [48]:
# dictionary of device models and their associated indices
device_categories = brand_df[['device_model']].drop_duplicates().reset_index(drop=True).reset_index()
device_categories = device_categories.rename(columns = {'index': 'device_model_id'})

In [49]:
brand_categories.head()

,phone_brand_id,phone_brand
0,0,小米
1,1,三星
2,2,SUGAR
3,3,华为
4,4,vivo


In [51]:
device_categories.head()

,device_model_id,device_model
0,0,红米
1,1,MI 2
2,2,Galaxy S4
3,3,时尚手机
4,4,Galaxy Note 2


In [53]:
# want a number for each phone-brand, not chinese name
brand_df = brand_df.merge(brand_categories, on='phone_brand')

In [54]:
brand_df = brand_df.merge(device_categories, on='device_model')

In [57]:
brand_df.drop(['device_model','phone_brand'], inplace=True, axis=1)

In [60]:
brand_df = brand_df.sort_values('device_id')

In [80]:
phone_brand_counts = brand_df.phone_brand_id.value_counts().rename_axis(
    'phone_brand_id').reset_index(name='counts').merge(brand_categories,
                                                       on='phone_brand_id')
phone_brand_counts.head(10) # top 10 phone brands

,phone_brand_id,counts,phone_brand
0,0,43210,小米
1,1,34286,三星
2,3,32564,华为
3,4,14395,vivo
4,8,14289,OPPO
5,5,11853,魅族
6,6,8407,酷派
7,11,6761,联想
8,10,2768,金立
9,15,2682,HTC


### Label Categories

In [116]:
display(label_categories.head())
display(app_labels.head())
display(app_events.head())
# can see that label_categories classifies the apps into categories
# app_labels just associates each app with their label
# app_events actually gives the app used with each event_id

,label_id,category
0,1,NaN
1,2,game-game type
2,3,game-Game themes
3,4,game-Art Style
4,5,game-Leisure time


,app_id,label_id
0,7324884708820027918,251
1,-4494216993218550286,251
2,6058196446775239644,406
3,6058196446775239644,407
4,8694625920731541625,406


,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [123]:
# look at number of app types and most common
print(label_categories.shape) # 930 types of apps
print(app_labels.shape) # 459,943 app downloads
common_categories = app_labels.label_id.value_counts().rename_axis(
    'label_id').reset_index(name='counts').merge(label_categories,
                                                       on='label_id')
common_categories.head(10)

(930, 2)
(459943, 2)


,label_id,counts,category
0,548,56902,Industry tag
1,405,53936,Custom label
2,794,49320,Tencent
3,795,48707,game
4,704,45697,Property Industry 2.0
5,714,19083,1 free
6,713,11840,Services 1
7,854,9955,Property Industry new
8,710,9027,Relatives 1
9,711,8831,Irritation / Fun 1
